In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.interpolate import CubicSpline
from sklearn.metrics import mean_squared_error
import warnings
import itertools
warnings.filterwarnings("ignore")
import arviz as az
import scipy
import matplotlib.patches as patches
import statsmodels.stats.api as sms
import scipy.stats.kde as kde

# Read 16S data

In [5]:
df_16S = pd.read_excel('../../../../../data/16S_relative_abundance.xlsx', index_col=0)
df_16S.columns = [c.replace('/','_slash_').replace(' ','_space_').replace('[','_leftsquarebracket_').replace(']','_rightsquarebracket_').replace('-','_dash_').replace('.','_dot_').replace('(','_leftroundbracket').replace(')','_rightroundbracket_') for c in df_16S.columns]
df_16S.head()

,A2,ASF356,Acetatifactor,Acetatifactor_dash_muris,Acidovorax,Acinetobacter,Acinetobacter_dash_sp_dot__dash_CIP_dash_56_dot_2,Acutalibacter_dash_muris,Adlercreutzia_dash_mucosicola,Adlercreutzia_dash_muris,...,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_ventriosum_dash_group,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_xylanophilum_dash_group,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_gnavus_dash_group,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_torques_dash_group,bacterium_dash_enrichment_dash_culture_dash_clone_dash_M153,gut_dash_metagenome,human_dash_gut_dash_metagenome,marine_dash_metagenome,metagenome,mouse_dash_gut_dash_metagenome
SampleID,,,,,,,,,,,,,,,,,,,,,
sample131,0.0,0.001183,0.000078,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000688,0.0,0.0,0.0,0.001803,0.0,0.0,0.0,0.001444
sample132,0.0,0.003461,0.000428,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000684,0.0,0.0,0.0,0.000910,0.0,0.0,0.0,0.008572
sample133,0.0,0.000619,0.000776,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.003797,0.0,0.0,0.0,0.003194
sample134,0.0,0.002272,0.000048,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000497,0.0,0.0,0.0,0.004312,0.0,0.0,0.0,0.004088
sample135,0.0,0.001034,0.000479,0.0,0.0,0.0,0.0,0.0,0.000142,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.006577,0.0,0.0,0.0,0.006032


# Add pseudocount for zeros

In [6]:
for sample_id in df_16S.index:
    sample = np.array(df_16S.loc[sample_id])
    minval = np.min(sample[np.nonzero(sample)]) # minimum non-zero value
    sample[sample==0] = minval
    df_16S.loc[sample_id] = sample
df_16S.head()

,A2,ASF356,Acetatifactor,Acetatifactor_dash_muris,Acidovorax,Acinetobacter,Acinetobacter_dash_sp_dot__dash_CIP_dash_56_dot_2,Acutalibacter_dash_muris,Adlercreutzia_dash_mucosicola,Adlercreutzia_dash_muris,...,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_ventriosum_dash_group,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_xylanophilum_dash_group,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_gnavus_dash_group,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_torques_dash_group,bacterium_dash_enrichment_dash_culture_dash_clone_dash_M153,gut_dash_metagenome,human_dash_gut_dash_metagenome,marine_dash_metagenome,metagenome,mouse_dash_gut_dash_metagenome
SampleID,,,,,,,,,,,,,,,,,,,,,
sample131,0.000010,0.001183,0.000078,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000688,0.000010,0.000010,0.000010,0.001803,0.000010,0.000010,0.000010,0.001444
sample132,0.000008,0.003461,0.000428,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,...,0.000008,0.000684,0.000008,0.000008,0.000008,0.000910,0.000008,0.000008,0.000008,0.008572
sample133,0.000008,0.000619,0.000776,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.003797,0.000008,0.000008,0.000008,0.003194
sample134,0.000014,0.002272,0.000048,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,...,0.000014,0.000497,0.000014,0.000014,0.000014,0.004312,0.000014,0.000014,0.000014,0.004088
sample135,0.000022,0.001034,0.000479,0.000022,0.000022,0.000022,0.000022,0.000022,0.000142,0.000022,...,0.000022,0.000022,0.000022,0.000022,0.000022,0.006577,0.000022,0.000022,0.000022,0.006032


# Read metadata for inulin and control

In [7]:
df_meta = pd.read_excel('../../../../../data/meta_data.xlsx', index_col=0)
df_meta = df_meta[(df_meta.Diet=='Inulin') | (df_meta.Diet=='Control')]
df_meta['Diet'] = [1 if g=='Inulin' else 0 for g in df_meta['Diet']]
df_meta.head()

,MiceID,Vendor,Day,Diet
SampleID,,,,
sample131,BC1,Beijing,0,0
sample132,BC2,Beijing,0,0
sample133,BC3,Beijing,0,0
sample134,BC4,Beijing,0,0
sample135,BC5,Beijing,0,0


# Select the 20 most abundant taxa

In [8]:
topX = 20
df_16S_T = df_16S.loc[df_meta.index].T
df_16S_T['mean'] = df_16S_T.mean(axis=1)
df_16S_T = df_16S_T.sort_values(by=['mean'],axis=0,ascending=False)
df_16S_T = df_16S_T.drop('mean', axis=1)
df_16S_topX = df_16S_T.iloc[0:topX].T
df_16S_topX.head()

,Muribaculaceae,Bacteroides,Faecalibaculum,Bacteroides_dash_acidifaciens,Parabacteroides_dash_goldsteinii,Parasutterella,Akkermansia_dash_muciniphila,Lachnospiraceae,Alloprevotella,Lachnospiraceae_dash_NK4A136_dash_group,Lachnospiraceae_dash_bacterium_dash_28_dash_4,Desulfovibrionaceae,Rikenellaceae_dash_RC9_dash_gut_dash_group,Oscillospiraceae,Clostridia_dash_vadinBB60_dash_group,Bacteroides_dash_uniformis,Clostridia_dash_UCG_dash_014,Alistipes,Helicobacter_dash_ganmani,Mucispirillum_dash_schaedleri
SampleID,,,,,,,,,,,,,,,,,,,,
sample131,0.237934,0.004178,0.237536,0.018370,0.001134,0.024167,0.041179,0.036914,0.106118,0.029702,0.001105,0.001987,0.074031,0.006824,0.005138,0.000756,0.001861,0.011623,0.000058,0.021927
sample132,0.217026,0.022603,0.148081,0.038074,0.000506,0.030693,0.022308,0.038992,0.002435,0.031797,0.067491,0.008470,0.029207,0.033602,0.001221,0.000856,0.000614,0.010656,0.000101,0.114954
sample133,0.236170,0.013355,0.213967,0.022921,0.000792,0.081351,0.063828,0.012851,0.017804,0.012587,0.003566,0.002575,0.086568,0.009055,0.002996,0.000850,0.000949,0.018225,0.000091,0.069399
sample134,0.181429,0.019500,0.116534,0.048061,0.000864,0.023371,0.106147,0.038334,0.017331,0.019330,0.000735,0.010890,0.108460,0.023867,0.000911,0.000592,0.000326,0.037851,0.000095,0.023860
sample135,0.225473,0.011760,0.218809,0.053997,0.000817,0.035780,0.053453,0.014144,0.020514,0.012380,0.000686,0.003082,0.120364,0.006152,0.000512,0.000283,0.006032,0.032165,0.000076,0.008613


In [9]:
list(df_16S_topX.columns)

['Muribaculaceae',
 'Bacteroides',
 'Faecalibaculum',
 'Bacteroides_dash_acidifaciens',
 'Parabacteroides_dash_goldsteinii',
 'Parasutterella',
 'Akkermansia_dash_muciniphila',
 'Lachnospiraceae',
 'Alloprevotella',
 'Lachnospiraceae_dash_NK4A136_dash_group',
 'Lachnospiraceae_dash_bacterium_dash_28_dash_4',
 'Desulfovibrionaceae',
 'Rikenellaceae_dash_RC9_dash_gut_dash_group',
 'Oscillospiraceae',
 'Clostridia_dash_vadinBB60_dash_group',
 'Bacteroides_dash_uniformis',
 'Clostridia_dash_UCG_dash_014',
 'Alistipes',
 'Helicobacter_dash_ganmani',
 'Mucispirillum_dash_schaedleri']

# Normalize max value of 16S data to 1

In [10]:
bacterial_taxa = list(df_16S_topX.columns)
df_16S_topX_w_meta = pd.merge(df_meta, df_16S_topX/df_16S_topX.max().max(), left_index=True, right_index=True, how='inner')
df_16S_topX_w_meta.head()

,MiceID,Vendor,Day,Diet,Muribaculaceae,Bacteroides,Faecalibaculum,Bacteroides_dash_acidifaciens,Parabacteroides_dash_goldsteinii,Parasutterella,...,Lachnospiraceae_dash_bacterium_dash_28_dash_4,Desulfovibrionaceae,Rikenellaceae_dash_RC9_dash_gut_dash_group,Oscillospiraceae,Clostridia_dash_vadinBB60_dash_group,Bacteroides_dash_uniformis,Clostridia_dash_UCG_dash_014,Alistipes,Helicobacter_dash_ganmani,Mucispirillum_dash_schaedleri
SampleID,,,,,,,,,,,,,,,,,,,,,
sample131,BC1,Beijing,0,0,0.318127,0.005586,0.317596,0.024561,0.001516,0.032312,...,0.001478,0.002657,0.098983,0.009125,0.006869,0.001011,0.002489,0.015540,0.000078,0.029318
sample132,BC2,Beijing,0,0,0.290174,0.030222,0.197990,0.050907,0.000676,0.041037,...,0.090239,0.011325,0.039051,0.044927,0.001633,0.001144,0.000822,0.014248,0.000135,0.153698
sample133,BC3,Beijing,0,0,0.315770,0.017856,0.286083,0.030647,0.001059,0.108770,...,0.004768,0.003443,0.115744,0.012106,0.004006,0.001137,0.001269,0.024367,0.000121,0.092790
sample134,BC4,Beijing,0,0,0.242578,0.026073,0.155810,0.064259,0.001155,0.031248,...,0.000982,0.014560,0.145016,0.031911,0.001219,0.000791,0.000437,0.050609,0.000127,0.031902
sample135,BC5,Beijing,0,0,0.301467,0.015723,0.292557,0.072197,0.001092,0.047840,...,0.000917,0.004120,0.160932,0.008226,0.000684,0.000379,0.008066,0.043006,0.000102,0.011516


# Calculate log-derivatives (dlog(B)/dt)

In [11]:
df_deriv = deepcopy(df_16S_topX_w_meta)
for curr_mice in set(df_deriv.MiceID):
    curr_df = df_deriv[df_deriv.MiceID==curr_mice]
    for taxon in bacterial_taxa:
        xdata = np.array(curr_df['Day'])
        ydata = np.array(curr_df[taxon])
        cs = CubicSpline(xdata, ydata)
        csd1 = cs.derivative(nu=1)
        ydata_d1 = csd1(xdata)
        df_deriv.loc[df_deriv.MiceID==curr_mice, taxon] = ydata_d1
df_deriv.head()

,MiceID,Vendor,Day,Diet,Muribaculaceae,Bacteroides,Faecalibaculum,Bacteroides_dash_acidifaciens,Parabacteroides_dash_goldsteinii,Parasutterella,...,Lachnospiraceae_dash_bacterium_dash_28_dash_4,Desulfovibrionaceae,Rikenellaceae_dash_RC9_dash_gut_dash_group,Oscillospiraceae,Clostridia_dash_vadinBB60_dash_group,Bacteroides_dash_uniformis,Clostridia_dash_UCG_dash_014,Alistipes,Helicobacter_dash_ganmani,Mucispirillum_dash_schaedleri
SampleID,,,,,,,,,,,,,,,,,,,,,
sample131,BC1,Beijing,0,0,-0.294894,0.198953,-0.687407,-0.019570,-0.000134,-0.080179,...,-0.054459,0.364591,0.008018,0.030366,-0.004121,0.002133,0.008565,0.080135,0.015186,-0.030793
sample132,BC2,Beijing,0,0,-0.369381,-0.022964,-0.527072,-0.083123,0.000630,-0.098706,...,-0.134347,0.572466,-0.002834,-0.010603,0.004008,-0.001277,0.000674,0.002221,0.203150,-0.188570
sample133,BC3,Beijing,0,0,-0.147410,0.191872,-0.541034,-0.073273,0.000083,-0.236399,...,-0.004334,0.216482,-0.016506,0.047448,0.060661,-0.001317,0.023582,0.172864,0.056731,-0.137101
sample134,BC4,Beijing,0,0,0.151784,-0.006264,-0.149190,-0.075447,-0.000307,-0.013529,...,0.005038,0.021675,0.026115,0.068254,0.016930,-0.001200,0.004031,-0.036927,0.000140,-0.035878
sample135,BC5,Beijing,0,0,0.156434,0.079137,-0.378469,-0.050401,-0.000142,-0.030689,...,0.017626,0.036313,-0.118671,0.015219,0.005966,0.000591,-0.007822,0.028085,-0.000087,-0.000541


# Construct regression matrix

In [12]:
# Ymat should be samples by bacteria
Ymat = df_deriv[bacterial_taxa].values
Ymat = Ymat.flatten(order='F')
Ymat = StandardScaler().fit_transform(Ymat.reshape(-1,1)).reshape(1,-1)[0] # standardize
Ymat

array([-5.38285709e+00, -6.74087977e+00, -2.69396521e+00, ...,
       -4.34048524e-02, -4.58747292e-04, -1.03068191e-02])

In [13]:
len(Ymat)

6740

In [14]:
Xmat = np.zeros(shape=(topX*len(df_deriv.index), (topX+2)*topX))
for k in np.arange(topX):
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)] = 1
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+1] = df_deriv.Diet.values
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+2:(k+1)*(topX+2)] = df_16S_topX_w_meta[bacterial_taxa].values
Xmat

array([[1.00000000e+00, 0.00000000e+00, 3.18127455e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 2.90173547e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 3.15770007e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.39557067e-04, 1.61027385e-04, 8.91018196e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.57211509e-04, 2.47685157e-04, 8.19266287e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        7.54193953e-05, 1.43296851e-04, 1.13129093e-04]])

# Write stan program

## write data

In [17]:
json_str = '{\n"N" : %d,\n'%(len(Ymat))

# y variable
json_str += '\"dlogX\" : [%s],\n'%(','.join(list(Ymat.astype(str))))

# x variable
for k1,c1 in enumerate(bacterial_taxa):
    # growth rate
    json_str += '\"growth_rate_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)].astype(str))))
    # diet response
    json_str += '\"inulin_response_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)+1].astype(str))))
    # bacterial interactions
    for k2,c2 in enumerate(bacterial_taxa):
        v = list(Xmat[:,k1*(topX+2)+2+k2].astype(str))
        json_str += '\"pairwise_interaction_%s_%s\" : [%s]'%(c1,c2,','.join(v))
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            json_str += '\n}'
        else:
            json_str += ',\n'
#print(json_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.data.json", "w")
text_file.write("%s" % json_str)
text_file.close()

## write model

In [18]:
# data block
model_str = 'data {\n'
model_str += '\tint<lower=0> N;\n'
model_str += '\tvector[N] dlogX;\n'
for c1 in bacterial_taxa:
    model_str += '\tvector[N] growth_rate_%s;\n'%(c1)
    model_str += '\tvector[N] inulin_response_%s;\n'%(c1)
    for c2 in bacterial_taxa:
        model_str += '\tvector[N] pairwise_interaction_%s_%s;\n'%(c1,c2)
model_str += '}\n'

# parameter block
model_str += 'parameters {\n\treal<lower=0,upper=1> sigma;\n'
for c1 in bacterial_taxa:
    model_str += '\treal alpha__%s;\n'%(c1) # growth rate
    model_str += '\treal epsilon__%s;\n'%(c1) # inulin response
    for c2 in bacterial_taxa:
        model_str += '\treal beta__%s_%s;\n'%(c1,c2)
model_str += '}\n'       
        
# model block
model_str += 'model {\n\tsigma ~ uniform(0,1);\n'
for c1 in bacterial_taxa:
    model_str += '\talpha__%s ~ normal(0,1);\n'%(c1) # growth rate
    model_str += '\tepsilon__%s ~ normal(0,1);\n'%(c1) # inulin response
    for c2 in bacterial_taxa:
        model_str += '\tbeta__%s_%s ~ normal(0,1);\n'%(c1,c2)
model_str += '\tdlogX ~ normal('
for c1 in bacterial_taxa:
    model_str += 'alpha__%s*growth_rate_%s+'%(c1,c1) # growth rate
    model_str += 'epsilon__%s*inulin_response_%s+'%(c1,c1) # inulin response
    for c2 in bacterial_taxa:
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s'%(c1,c2,c1,c2)
        else:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s+'%(c1,c2,c1,c2)
model_str += ', sigma);\n}'
#print(model_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.stan", "w")
text_file.write("%s" % model_str)
text_file.close()

# Plot stan output

In [15]:
def hpd_grid(sample, alpha=0.05, roundto=2):
    """Calculate highest posterior density (HPD) of array for given alpha. 
    The HPD is the minimum width Bayesian credible interval (BCI). 
    The function works for multimodal distributions, returning more than one mode
    Parameters
    ----------
    
    sample : Numpy array or python list
        An array containing MCMC samples
    alpha : float
        Desired probability of type I error (defaults to 0.05)
    roundto: integer
        Number of digits after the decimal point for the results
    Returns
    ----------
    hpd: array with the lower 
          
    """
    sample = np.asarray(sample)
    sample = sample[~np.isnan(sample)]
    # get upper and lower bounds
    l = np.min(sample)
    u = np.max(sample)
    density = kde.gaussian_kde(sample)
    x = np.linspace(l, u, 2000)
    y = density.evaluate(x)
    #y = density.evaluate(x, l, u) waitting for PR to be accepted
    xy_zipped = zip(x, y/np.sum(y))
    xy = sorted(xy_zipped, key=lambda x: x[1], reverse=True)
    xy_cum_sum = 0
    hdv = []
    for val in xy:
        xy_cum_sum += val[1]
        hdv.append(val[0])
        if xy_cum_sum >= (1-alpha):
            break
    hdv.sort()
    diff = (u-l)/20  # differences of 5%
    hpd = []
    hpd.append(round(min(hdv), roundto))
    for i in range(1, len(hdv)):
        if hdv[i]-hdv[i-1] >= diff:
            hpd.append(round(hdv[i-1], roundto))
            hpd.append(round(hdv[i], roundto))
    hpd.append(round(max(hdv), roundto))
    ite = iter(hpd)
    hpd = list(zip(ite, ite))
    modes = []
    for value in hpd:
        x_hpd = x[(x > value[0]) & (x < value[1])]
        y_hpd = y[(x > value[0]) & (x < value[1])]
        modes.append(round(x_hpd[np.argmax(y_hpd)], roundto))
    return hpd, x, y, modes

In [3]:
fit = az.from_cmdstan(["output_%d.csv"%(i) for i in np.arange(1,4)])

In [16]:
lines = []
    
# basal growth rate
for c in bacterial_taxa:
    var = 'alpha__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['basal_growth_rate', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
# inulin response
for c in bacterial_taxa:
    var = 'epsilon__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['inulin_response', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])

# pairwise interactions
for c1 in bacterial_taxa:
    for c2 in bacterial_taxa:
        var = 'beta__%s_%s'%(c1,c2)
        data = []
        for i in np.arange(0,3):
            data.extend(list(fit.posterior[var][i].values))
        hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
        assert len(hpd_mu) == 1
        (x0, x1) = hpd_mu[0]
        lines.append(['pairwise_interaction', (c1,c2), x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
df_stan_output_summary = pd.DataFrame(lines, columns = ['Type','Taxa','Left','Right','Middle','SNR','Significant'])
df_stan_output_summary.head()

,Type,Taxa,Left,Right,Middle,SNR,Significant
0,basal_growth_rate,Muribaculaceae,-0.64,0.29,-0.175,-0.768734,False
1,basal_growth_rate,Bacteroides,-0.29,0.64,0.175,0.732938,False
2,basal_growth_rate,Faecalibaculum,-0.39,0.55,0.080,0.348288,False
3,basal_growth_rate,Bacteroides_dash_acidifaciens,0.39,1.33,0.860,3.619062,True
4,basal_growth_rate,Parabacteroides_dash_goldsteinii,-0.55,0.39,-0.080,-0.361593,False


In [17]:
df_stan_output_summary.to_excel('bayesian_regression_summary_top20.xlsx')

# Plot

In [18]:
df_stan_output_summary = pd.read_excel('bayesian_regression_summary_top20.xlsx', index_col=0)
df_stan_output_summary = df_stan_output_summary[df_stan_output_summary.Significant==True]
df_stan_output_summary

,Type,Taxa,Left,Right,Middle,SNR,Significant
3,basal_growth_rate,Bacteroides_dash_acidifaciens,0.39,1.33,0.860,3.619062,True
22,inulin_response,Faecalibaculum,0.63,1.22,0.925,6.199610,True
23,inulin_response,Bacteroides_dash_acidifaciens,2.02,2.60,2.310,15.578801,True
25,inulin_response,Parasutterella,0.21,0.80,0.505,3.377636,True
28,inulin_response,Alloprevotella,-0.93,-0.34,-0.635,-4.281057,True
31,inulin_response,Desulfovibrionaceae,-0.91,-0.32,-0.615,-4.114519,True
37,inulin_response,Alistipes,-0.60,-0.01,-0.305,-2.046231,True
38,inulin_response,Helicobacter_dash_ganmani,-0.60,-0.01,-0.305,-2.003188,True
40,pairwise_interaction,"('Muribaculaceae', 'Muribaculaceae')",0.11,1.44,0.775,2.317362,True
51,pairwise_interaction,"('Muribaculaceae', 'Desulfovibrionaceae')",0.37,2.96,1.665,2.552528,True
